In [1]:
# imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = 'plotly'

# data
## stores

In [2]:

df_store = pd.read_pickle("data/df_daily.pkl")

# store_id as index
# mean sales
# drop promo_count
df_store_1 = df_store.groupby("store_id").mean().drop("promo_count", axis=1)[["sales"]]

# keep the rest
df_store_2 = df_store.groupby("store_id").first().loc[:, "store_level":]

# count number of operating days
df_store_3 = df_store.groupby("store_id").count()[["date"]]

# join
df = df_store_1.join(df_store_2).join(df_store_3)
df["sales"] = df["sales"] / 1e6
df = df.rename(columns={
    "sales": "sales_daily_mean", 
    "date": "oper_duration",
    })

idx = []
for i in df.index:
    idx.append('store_' + str(i))

df.index = idx
df.head()


,sales_daily_mean,store_level,store_group,store_format,store_segment,opening_date,status,store_area,number_of_staff,province,channel,oper_duration
store_307222,24.748964,A,Trực thuộc,Phố,Mainstream,2011-01-11,Active,112.0,6.0,Hồ Chí Minh,Retail,1274
store_307244,19.275418,A,Trực thuộc,Phố,Mainstream,2013-04-25,Active,170.0,5.0,Hồ Chí Minh,Retail,1271
store_307248,16.027751,B,Trực thuộc,Phố,Mass,2013-11-07,Active,78.0,4.0,Hồ Chí Minh,Retail,1264
store_320264,14.814896,B,Trực thuộc,Phố,Mass,2013-10-01,Active,120.0,5.0,Hồ Chí Minh,Retail,1256
store_328165,44.437752,A,Trực thuộc,Mall,Flagship,2019-04-25,Active,380.0,14.0,Hồ Chí Minh,Retail,1272


## scores

In [8]:

SNAIVE = []
for i in [7,14,21,28]:
    p = "results/f8/SNAIVE_result_" + str(i) + ".pkl"
    SNAIVE.append(pd.read_pickle(p).set_index("store"))
SNAIVE = pd.concat(SNAIVE, axis=1, keys=['7','14','21','28'])

ARIMA = []
for i in [7,14,21,28]:
    p = "results/f8/ARIMA_result_" + str(i) + ".pkl"
    ARIMA.append(pd.read_pickle(p).set_index("store"))
ARIMA = pd.concat(ARIMA, axis=1, keys=['7','14','21','28'])

PROPHET = []
for i in [7,14,21,28]:
    p = "results/f8/PROPHET_result_" + str(i) + ".pkl"
    PROPHET.append(pd.read_pickle(p).set_index("store"))
PROPHET = pd.concat(PROPHET, axis=1, keys=['7','14','21','28'])

RF = []
for i in [7,14,21,28]:
    p = "results/f8/RF_result_" + str(i) + ".pkl"
    RF.append(pd.read_pickle(p).set_index("store"))
RF = pd.concat(RF, axis=1, keys=['7','14','21','28'])

XGB = []
for i in [7,14,21,28]:
    p = "results/f8/XGB_result_" + str(i) + ".pkl"
    XGB.append(pd.read_pickle(p).set_index("store"))
XGB = pd.concat(XGB, axis=1, keys=['7','14','21','28'])


In [33]:
df_score = pd.concat([SNAIVE, ARIMA, PROPHET, RF, XGB], axis=1)
df_score.head(2)

7             \
                                                          sales mae_SNAIVE   
store                                                                        
store_307222  date
2021-01-25    24.848540
2021-01-26    18....    7.82875   
store_307244  date
2021-01-25    17.889600
2021-01-26    18....    5.05825   

                                                  \
             rmse_SNAIVE mape_SNAIVE mase_SNAIVE   
store                                              
store_307222    10.07600      0.2515     0.50900   
store_307244     6.30275      0.2595     0.39525   

                                                                 \
                                                      fc_SNAIVE   
store                                                             
store_307222  2021-01-25    17.433143
2021-01-26    19.11446...   
store_307244  2021-01-25    14.368380
2021-01-26    14.75122...   

                                                             14             \
                                                          sales mae_SNAIVE   
store                                                                        
store_307222  date
2021-01-18    18.799600
2021-01-19    27....     7.8180   
store_307244  date
2021-01-18    10.078400
2021-01-19    17....     5.4435   

                                      ...       21                             \
             rmse_SNAIVE mape_SNAIVE  ...  mae_XGB rmse_XGB mape_XGB mase_XGB   
store                                 ...                                       
store_307222    10.00625      0.2885  ...  2.18950  3.09975  0.07925   0.2240   
store_307244     6.82700      0.2950  ...  1.52925  2.07575  0.06825   0.1965   

                                                                      28  \
                                                         fc_XGB  mae_XGB   
store                                                                      
store_307222  date
2021-01-11    15.724278
2021-01-12    21....  3.51325   
store_307244  date
2021-01-11     9.004819
2021-01-12    15....  1.96375   

                                         \
             rmse_XGB mape_XGB mase_XGB   
store                                     
store_307222  4.52325  0.12975   0.3605   
store_307244  2.89650  0.09775   0.2520   

                                                                 
                                                         fc_XGB  
store                                                            
store_307222  date
2021-01-04    14.784580
2021-01-05    18....  
store_307244  date
2021-01-04    12.348523
2021-01-05    12....  

[2 rows x 104 columns]

# horizon = 7

In [370]:
# load data
score_SNAIVE_7 = pd.read_pickle("results/f8/SNAIVE_result_7.pkl").set_index("store")
score_ARIMA_7 = pd.read_pickle("results/f8/ARIMA_result_7.pkl").set_index("store")
score_PROPHET_7 = pd.read_pickle("results/f8/PROPHET_result_7.pkl").set_index("store")
score_RF_7 = pd.read_pickle("results/f8/RF_result_7.pkl").set_index("store")
score_XGB_7 = pd.read_pickle("results/f8/XGB_result_7.pkl").set_index("store")

score_ALL_7 = score_SNAIVE_7.join(score_ARIMA_7).join(score_PROPHET_7).join(score_RF_7).join(score_XGB_7).join(df)
score_ALL_7.head(3)

,sales,mae_SNAIVE,rmse_SNAIVE,mape_SNAIVE,mase_SNAIVE,fc_SNAIVE,mae_ARIMA,rmse_ARIMA,mape_ARIMA,mase_ARIMA,...,store_group,store_format,store_segment,opening_date,status,store_area,number_of_staff,province,channel,oper_duration
store,,,,,,,,,,,,,,,,,,,,,
store_307222,date 2021-01-25 24.848540 2021-01-26 18....,7.82875,10.07600,0.2515,0.50900,2021-01-25 17.433143 2021-01-26 19.11446...,7.91225,9.09375,0.2810,0.80050,...,Trực thuộc,Phố,Mainstream,2011-01-11,Active,112.0,6.0,Hồ Chí Minh,Retail,1274
store_307244,date 2021-01-25 17.889600 2021-01-26 18....,5.05825,6.30275,0.2595,0.39525,2021-01-25 14.368380 2021-01-26 14.75122...,6.67450,7.74325,0.3455,0.85575,...,Trực thuộc,Phố,Mainstream,2013-04-25,Active,170.0,5.0,Hồ Chí Minh,Retail,1271
store_307248,date 2021-01-25 21.487100 2021-01-26 23....,7.88775,9.26900,0.3830,0.75725,2021-01-25 12.218554 2021-01-26 12.39495...,7.31325,8.78150,0.3630,1.06350,...,Trực thuộc,Phố,Mass,2013-11-07,Active,78.0,4.0,Hồ Chí Minh,Retail,1264


In [336]:
score_ALL_7.columns

Index(['sales', 'mae_SNAIVE', 'rmse_SNAIVE', 'mape_SNAIVE', 'mase_SNAIVE',
       'fc_SNAIVE', 'mae_ARIMA', 'rmse_ARIMA', 'mape_ARIMA', 'mase_ARIMA',
       'fc_ARIMA', 'mae_PROPHET', 'rmse_PROPHET', 'mape_PROPHET',
       'mase_PROPHET', 'fc_PROPHET', 'mae_RF', 'rmse_RF', 'mape_RF', 'mase_RF',
       'fc_RF', 'mae_XGB', 'rmse_XGB', 'mape_XGB', 'mase_XGB', 'fc_XGB',
       'sales_daily_mean', 'store_level', 'store_group', 'store_format',
       'store_segment', 'opening_date', 'status', 'store_area',
       'number_of_staff', 'province', 'channel', 'oper_duration'],
      dtype='object')

## MAPE & MASE

In [334]:
# mape
score_ALL_7.describe().filter(items=['mean','std'], axis=0).filter(like='mape', axis=1)


,mape_SNAIVE,mape_ARIMA,mape_PROPHET,mape_RF,mape_XGB
mean,0.418559,0.428039,0.338375,0.106066,0.108197
std,0.114154,0.092490,0.098701,0.043175,0.040826


In [335]:
# mase
score_ALL_7.describe().filter(items=['mean','std'], axis=0).filter(like='mase', axis=1)


,mase_SNAIVE,mase_ARIMA,mase_PROPHET,mase_RF,mase_XGB
mean,0.898513,1.298664,0.951572,0.298632,0.271566
std,0.437712,0.338098,0.203832,0.149338,0.095334


## Visualize

### Scores

In [351]:
score_ALL_7.filter(like='mae', axis=1).join(score_ALL_7[['sales_daily_mean']]).sort_values('sales_daily_mean').plot()


In [342]:
fig = score_ALL_7.filter(like='mae', axis=1).plot()
# fig.update_traces(marker=dict(color="RoyalBlue"))
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='MAE')


In [279]:
type(fig)

plotly.graph_objs._figure.Figure

In [271]:
fig = score_ALL_7.filter(like='rmse', axis=1).plot()
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='RMSE')


In [272]:
fig = score_ALL_7.filter(like='mape', axis=1).plot()
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='MAPE')


In [273]:
fig = score_ALL_7.filter(like='mase', axis=1).plot()
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='MASE')


### Forecasts

In [274]:
sample_ls = ['store_307222', 'store_350018', 'store_566790']

y_true = [df_store.loc[df_store['store_id']==307222, 'sales'][-7:].values,
          df_store.loc[df_store['store_id']==350018, 'sales'][-7:].values,
          df_store.loc[df_store['store_id']==566790, 'sales'][-7:].values]
y_true=pd.DataFrame({'sales':y_true}, index=sample_ls)

sample_fc = score_ALL_7.filter(items=sample_ls, axis=0).filter(like='fc', axis=1).join(y_true)
sample_fc


,fc_SNAIVE,fc_ARIMA,fc_PROPHET,fc_RF,fc_XGB,sales
store,,,,,,
store_307222,2021-01-25 17.433143 2021-01-26 19.11446...,"[48.20032137436968, 42.39733446660982, 38.4800...",ds trend yhat_lower yhat_uppe...,date 2021-01-25 28.435211 2021-01-26 17....,date 2021-01-25 27.353668 2021-01-26 18....,"[24.84854, 18.834033, 31.418629, 27.555219, 35..."
store_350018,2021-01-25 16.534347 2021-01-26 15.54449...,"[75.80788006524239, 67.12306191270103, 65.1767...",ds trend yhat_lower yhat_uppe...,date 2021-01-25 39.526892 2021-01-26 4...,date 2021-01-25 40.666094 2021-01-26 4...,"[47.7719, 38.8677, 30.6969, 47.07535, 56.7408,..."
store_566790,2021-01-25 17.134802 2021-01-26 16.46452...,"[30.710727370440367, 25.258218878968346, 22.29...",ds trend yhat_lower yhat_uppe...,date 2021-01-25 30.229686 2021-01-26 14....,date 2021-01-25 28.866780 2021-01-26 19....,"[26.1868, 19.6689, 26.766327, 34.1142, 44.4857..."


In [275]:
store = sample_ls[0]
x = sample_fc.loc[store,'fc_SNAIVE'].index

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'sales'], 
                         name='Actual sales',
                         line={'color': 'grey'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_SNAIVE'].values, 
                         name='SNAIVE',
                         line={'color': 'salmon', 'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_ARIMA'], 
                         name='SARIMAX',
                         line={'color': 'dodgerblue', 'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_PROPHET']['yhat'].values, 
                         name='PROPHET',
                         line={'color': 'slateblue', 'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_RF'].values, 
                         name='Random Forest',
                         line={'color': 'hotpink', 'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_XGB'].values, 
                         name='XGBoost',
                         line={'color': 'forestgreen', 'dash': 'dash'},
                         ))
fig.update_layout(title='Sales Forecast ' + store)
fig.update_yaxes(title_text='Forecasts (million VND)')

fig.show()


In [276]:
store = sample_ls[1]
x = sample_fc.loc[store,'fc_SNAIVE'].index

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'sales'], 
                         name='Actual sales',
                         line={'color': 'grey'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_SNAIVE'].values, 
                         name='SNAIVE',
                         line={'color': 'salmon', 'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_ARIMA'], 
                         name='SARIMAX',
                         line={'color': 'dodgerblue', 'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_PROPHET']['yhat'].values, 
                         name='PROPHET',
                         line={'color': 'slateblue', 'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_RF'].values, 
                         name='Random Forest',
                         line={'color': 'hotpink', 'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_XGB'].values, 
                         name='XGBoost',
                         line={'color': 'forestgreen', 'dash': 'dash'},
                         ))

fig.update_layout(title='Sales Forecast ' + store)
fig.update_yaxes(title_text='Forecasts (million VND)')

fig.show()


In [277]:
store = sample_ls[2]
x = sample_fc.loc[store,'fc_SNAIVE'].index

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'sales'], 
                         name='Actual sales',
                         line={'color': 'grey'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_SNAIVE'].values, 
                         name='SNAIVE',
                         line={'color': 'salmon', 'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_ARIMA'], 
                         name='SARIMAX',
                         line={'color': 'dodgerblue', 'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_PROPHET']['yhat'].values, 
                         name='PROPHET',
                         line={'color': 'slateblue', 'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_RF'].values, 
                         name='Random Forest',
                         line={'color': 'hotpink', 'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_XGB'].values, 
                         name='XGBoost',
                         line={'color': 'forestgreen', 'dash': 'dash'},
                         ))

fig.update_layout(title='Sales Forecast ' + store)
fig.update_yaxes(title_text='Forecasts (million VND)')

fig.show()


# horizon = 14

In [363]:
# load data
score_SNAIVE_14 = pd.read_pickle("results/f8/SNAIVE_result_14.pkl").set_index("store")
score_ARIMA_14 = pd.read_pickle("results/f8/ARIMA_result_14.pkl").set_index("store")
score_PROPHET_14 = pd.read_pickle("results/f8/PROPHET_result_14.pkl").set_index("store")
score_RF_14 = pd.read_pickle("results/f8/RF_result_14.pkl").set_index("store")
score_XGB_14 = pd.read_pickle("results/f8/XGB_result_14.pkl").set_index("store")

score_ALL_14 = score_SNAIVE_14.join(score_ARIMA_14).join(score_PROPHET_14).join(score_RF_14).join(score_XGB_14).join(df)

## MAPE & MASE

In [295]:
# mape
score_ALL_14.describe().filter(items=['mean','std'], axis=0).filter(like='mape', axis=1)


,mape_SNAIVE,mape_ARIMA,mape_PROPHET,mape_RF,mape_XGB
mean,0.413770,0.415862,0.338375,0.131297,0.092967
std,0.083252,0.078126,0.098701,0.069535,0.045135


In [296]:
# mase
score_ALL_14.describe().filter(items=['mean','std'], axis=0).filter(like='mase', axis=1)


,mase_SNAIVE,mase_ARIMA,mase_PROPHET,mase_RF,mase_XGB
mean,0.799382,0.997480,0.951572,0.295981,0.210283
std,0.350386,0.216448,0.203832,0.101431,0.088034


## Visualize

In [364]:
score_ALL_14.filter(like="mae", axis=1).join(
    score_ALL_14[["sales_daily_mean"]]
).sort_values("sales_daily_mean").plot()


In [365]:
fig = score_ALL_14.filter(like='mae', axis=1).plot()
# fig.update_traces(marker=dict(color="RoyalBlue"))
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='MAE')


In [366]:
fig = score_ALL_14.filter(like='rmse', axis=1).plot()
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='RMSE')


In [367]:
fig = score_ALL_14.filter(like='mape', axis=1).plot()
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='MAPE')


In [368]:
fig = score_ALL_14.filter(like='mase', axis=1).plot()
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='MASE')


# compare scores of different horizons

## MAPE & MASE

In [63]:
# mean mape of all models & all horizons
mean_mape = df_score.iloc[:4,:].describe().filter(items=['mean'], axis=0).filter(like='mape', axis=1).stack().T.droplevel(0,1)
mean_mape.index = mean_mape.index.astype('int')
mean_mape.sort_index(inplace=True)

mean_mape


,mape_ARIMA,mape_PROPHET,mape_RF,mape_SNAIVE,mape_XGB
7,0.345125,0.262687,0.069187,0.299875,0.071375
14,0.338688,0.262687,0.081018,0.328688,0.056750
21,0.451375,0.268062,0.119750,0.375250,0.088938
28,0.417625,0.269875,0.180438,0.363875,0.119500


In [62]:
fig = mean_mape.plot()
fig.update_xaxes(tick0=7, dtick=7, title_text='horizons')
fig.update_yaxes(title_text='mape')


In [61]:
# mean mase of all models & all horizons
mean_mase = df_score.describe().filter(items=['mean'], axis=0).filter(like='mase', axis=1).stack().T.droplevel(0,1)
mean_mase.index = mean_mase.index.astype('int')
mean_mase.sort_index(inplace=True)

fig = mean_mase.plot()
fig.update_xaxes(tick0=7, dtick=7, title_text='horizons')
fig.update_yaxes(title_text='mase')
